# Scraping and Analysis Tasks

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/)
* Collect the latest News Title and Paragraph Text
* Assign the text to variables that you can reference later.

### Data Sources: 

   #### All the data were scraped from the following websites:

* [NASA Mars News Site](https://mars.nasa.gov/news/) -Scraped the latest News Title and Paragraph Text
* [JPL Featured Space Image](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars) - Scraped the image url for the current Featured Mars Image
* [Mars Weather twitter account](https://twitter.com/marswxreport?lang=en) - Scraped the latest Mars weather tweet
* [USGS Astrogeology site](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) - Scraped high resolution images for each of Mar's hemispheres
* [Mars Facts webpage](https://space-facts.com/mars/) -Scraped the table containing facts about the planet including Diameter, Mass, etc

In [54]:
# Import Dependencies & Setup
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
from selenium import webdriver
import pandas as pd
import requests
import time
import pymongo

### Window users (long and old way)

In [55]:
# Set Executable Path & Initialize Chrome Browser

# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

### MAC users  (long and old way)

In [56]:
# Set Executable Path & Initialize Chrome Browser
# executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
# browser = Browser("chrome", **executable_path, headless=False)

### For Chrome

In [68]:
# Setup splinter: Looks/Scrape throu many pages instead of just one page 
# In order to run our selenium webdriver automation scripts on chrome and firefox browser, need to have browser drivers
executable_path = {'executable_path': ChromeDriverManager().install()} 
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\cache\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


### For FireFox

In [69]:
# from webdriver_manager.firefox import GeckoDriverManager
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# NASA Mars News

* Scrape the NASA Mars News Site and collect the latest:
    * News Title
    * Paragraph Text 
* Assign the text to variables that you can reference later.

In [70]:
# Set up url
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)

In [71]:
# Parse Results HTML with BeautifulSoup
nasa_html = browser.html
nasa_soup = BeautifulSoup(nasa_html, 'html.parser')

In [79]:
# Scrape HTML & Finding Everything inside ul
nasa_results = nasa_soup.find('ul', class_='item_list')

#nasa_results

### Scrape and collect the latest News "TITLE"

In [80]:
news_title = nasa_results.find("div", class_="content_title").text
print(news_title)

MOXIE Could Help Future Rockets Launch Off Mars


###  Scrape and collect the "Paragraph Text"

In [ ]:
news_parag = nasa_results.find("div", class_="content_title").text
print(news_parag)